In [1]:
import pandas as pd

In [ ]:
import os

In [ ]:
filelise = os.listdir('Downloads/play')

In [ ]:
# deal with '()' and '-' in song names
def formalize(string):
    string = str(string)
    if('-' in string):
        return(string[:string.find('-')])
    elif('(' in string):
        return(string[:string.find('(')])
    elif('（' in string):
        return(string[:string.find('（')])
    else:
        return string.strip()

def not_empty(string):
    if(string.strip()==''):
        return False
    else:
        return True
    
def retain_Float(string):
    if('.0' in string) or (string.isdigit()):
        return string
    else:
        return ''

import re
def isnumber(num):
    regex = re.compile(r"^(\d+)(\.\d*)?$")
    if re.match(regex,num):
        return True
    else:
        return False
    
def check_device(string):
    if(string == 'ar') or (string == 'ip') or (string == 'wp'):
        return True
    else:
        return False

def check_filename(string):
    if 'play.log' in string:
        return True
    else:
        return False
    
def get_date(file_name):
    tmp_list = str(file_name).split('_')
    return tmp_list[0]

def impute_length(number):
    if number == 0:
        return 245.0
    else:
        return number

def reasonable(number):
    if number > 28800:
        return False
    else:
        return True

In [ ]:
# read the file one by one and do manipulation
names = []
a = 0
b = 0
for i in filelise:   
    if 'fn' in i:
        a+= 1
        print(i)
        try:
            df = pd.read_csv('Downloads/play/' + i,delimiter='\t',header=None,index_col=None,names=schema,encoding ='utf-8')
        except UnicodeDecodeError:
            print(i+' has mistake')
            b+=1
            continue
        # retain entries with notnull songnames
        df = df[df['song_name'].notnull()].copy()
        df['song_name_new'] = df['song_name'].map(formalize)
        df = df[df['song_name_new'].map(not_empty)]
        
        # deal with abnormal userid
        df['uid'] = df['uid'].astype(str).map(retain_Float)
        df = df[df['uid']!=''].copy()
        df['uid'] = df['uid'].astype(float).astype(int)
        
        # device has three types: android iphone and windows phone
        df = df[df['device'].astype(str).map(check_device)]
        
        # filename must have 'play.log' as substring
        df = df[df['file_name'].astype(str).map(check_filename)]
        
        df['date'] = df['file_name'].map(get_date)
        
        # set all NaN to 0 in play_time and song_length
        df['song_length'] = df['song_length'].fillna(0)
        df['play_time'] = df['play_time'].fillna(0)
        
        # retain only numeric value
        df['song_length'] = df['song_length'].astype(str)
        df['play_time'] = df['play_time'].astype(str)
        df = df[df['song_length'].map(isnumber)]
        df = df[df['play_time'].map(isnumber)]
        df['song_length'] = df['song_length'].astype(float)
        df['play_time'] = df['play_time'].astype(float)
        df = df[df['play_time'].map(reasonable)]
        df['song_length_new'] = df['song_length'].map(impute_length)
        df['counts'] = df['play_time']/df['song_length_new']
        
        
        names.append(df['song_name_new'])

        output = df[['uid','song_name_new','counts']]
        output.to_csv('Downloads/justdata/'+i, sep=',', encoding='utf-8',index=False,header=False)

print("%s total %s abnormal" %(a,b))

In [ ]:
# get the song names
total_songs = set()
for i in names:
    total_songs = total_songs.union(set(i.unique()))
total = list(total_songs)

In [ ]:
len(total_songs)

In [ ]:
import pickle

In [ ]:
# save the long list
output = open('Downloads/data.pkl', 'wb')
pickle.dump(total, output, -1)

In [ ]:
pkl_file = open('Downloads/data.pkl', 'rb')
total = pickle.load(pkl_file)

In [ ]:
# make a dictionary of songnames for more effeciently search
total_dic = {}
n=0
for i in total:
    total_dic[i]=n
    n+=1

In [ ]:
# cast all song names to int
schema = ['uid','song_name','counts']
filelist = os.listdir('Downloads/justdata')
def name_to_int(string):
    return total_dic[string]

for i in filelist:
    print(i)
    df = pd.read_csv('Downloads/justdata/' + i,delimiter=',',header=None,index_col=None,names= schema,encoding ='utf-8')
    df['songid'] = df['song_name'].map(name_to_int)
    output = df[['uid','songid','counts']]
    output.to_csv('Downloads/finaldata/'+i, sep=',', encoding='utf-8',index=False,header=False)

In [ ]:
# after concatenate all files into one in command line
schema = ['usid','songid','counts']
df = pd.read_csv('Downloads/all_play.log.fn' ,delimiter=',',header=None,index_col=None,names=schema,encoding ='utf-8')
sum_count = df['counts'].groupby([df['usid'],df['songid']]).sum()
a = sum_count.reset_index()

In [ ]:
# cast counts to explicit rate
def to_rate(number):
    if number<1:
        return 1
    elif number<=5:
        return 2
    elif number<=30:
        return 3
    elif number <=100:
        return 4
    else:
        return 5
a['rating'] = a['counts'].map(to_rate)

In [ ]:
# save the data 
a[['usid','songid','rating']].to_csv('Downloads/ratingdata', sep=',', encoding='utf-8',index=False,header=False)